# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science 
## Homework 1 - Smoothers and Generalized Additive Models



**Harvard University**<br/>
**Spring 2020**<br/>
**Instructors**: Mark Glickman, Pavlos Protopapas, & Chris Tanner


<hr style="height:2pt">

### Homework 1 is due February 6th

In [1]:
#PLEASE RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment, please follow the instructions on Canvas.
- Please restart the kernel and run the entire notebook again before you submit.

<hr style="height:2pt">

### Please use the libraries below:

In [25]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from pygam import LinearGAM, s, f
from sklearn.preprocessing import LabelEncoder

# Seaborn visualization library
import seaborn as sns


<hr style="height:2pt">

### Problem 1 - Modeling Seasonality of Airbnb Prices
In this problem, the task is to build a regression model to predict the price of an Airbnb rental for a given date. The data are provided in `calendar_train.csv` and `calendar_test.csv`, which contain availability and price data for a sample of Airbnb units in the Boston area from 2017 to 2018, about 4 observations per day in each set.

Start by loading the data using pandas.  
*Hint*: You likely want to have pandas parse the `date` column as a datetime object via the `parse_dates` argument of `pd.read_csv`

In [26]:
# del before submission

import os

os.getcwd()

'/Users/gurdeep.singh/Desktop/OneDrive - Tata Communications/Harvard Courses/CS109B_2020/hw1'

In [27]:
abnb_train = pd.read_csv("data/calendar_train.csv", parse_dates = ['date'])
abnb_train.head()

,listing_id,date,available,price
0,20872145,2018-09-21,f,NaN
1,20872145,2018-09-19,f,NaN
2,20872145,2018-09-18,f,NaN
3,20872145,2018-09-17,f,NaN
4,20872145,2018-09-16,f,NaN


In [28]:
print("Shape: ", abnb_train.shape)
abnb_train.describe() # different values for count of listing_id and price

('Shape: ', (734003, 4))


,listing_id,price
count,7.340030e+05,309288.000000
mean,1.253864e+07,238.780612
std,6.334971e+06,188.527983
min,3.781000e+03,15.000000
25%,7.281884e+06,115.000000
50%,1.390864e+07,190.000000
75%,1.835457e+07,299.000000
max,2.122836e+07,5993.000000


In [29]:
abnb_train.dropna(inplace = True)
print("Shape: ", abnb_train.shape)
abnb_train.describe() # count is now identical

('Shape: ', (309288, 4))


,listing_id,price
count,3.092880e+05,309288.000000
mean,1.185732e+07,238.780612
std,6.716094e+06,188.527983
min,3.781000e+03,15.000000
25%,5.940866e+06,115.000000
50%,1.302981e+07,190.000000
75%,1.815527e+07,299.000000
max,2.122836e+07,5993.000000


In [30]:
abnb_train['day'] = abnb_train['date'].dt.weekday_name
abnb_train.head()

,listing_id,date,available,price,day
123,20872145,2018-04-02,t,62.0,Monday
124,20872145,2018-04-01,t,59.0,Sunday
125,20872145,2018-03-31,t,75.0,Saturday
126,20872145,2018-03-30,t,71.0,Friday
127,20872145,2018-03-28,t,51.0,Wednesday


In [31]:
abnb_test = pd.read_csv("data/calendar_test.csv", parse_dates = ['date'])
print("Shape: ", abnb_test.shape)
abnb_test.head()

('Shape: ', (314572, 4))


,listing_id,date,available,price
0,21205442,2018-09-28,t,138.0
1,5166870,2018-08-11,t,210.0
2,9698823,2017-10-17,f,NaN
3,18894466,2018-02-21,f,NaN
4,6765855,2018-09-22,f,NaN


In [32]:
abnb_test.dropna(inplace = True)
print("Shape: ", abnb_test.shape)


('Shape: ', (133100, 4))


In [33]:
abnb_test['day'] = abnb_test['date'].dt.weekday_name
abnb_test.head()

,listing_id,date,available,price,day
0,21205442,2018-09-28,t,138.0,Friday
1,5166870,2018-08-11,t,210.0,Saturday
12,19455818,2018-04-13,t,869.0,Friday
14,20351854,2017-12-23,t,239.0,Saturday
17,20622324,2018-02-09,t,259.0,Friday


### Exploratory Analysis

Visualize the average price by month and day of the week (i.e., Monday, Tuesday, etc.) for the training set. Point out any trends you notice and explain whether or not they make sense.

In [57]:
abnb_train['year'] = abnb_train['date'].map(lambda x: x.strftime('%Y'))
abnb_train.head(n=2)


,listing_id,date,available,price,day,year
123,20872145,2018-04-02,t,62.0,Monday,2018
124,20872145,2018-04-01,t,59.0,Sunday,2018


In [111]:
abnbtr_weekly = abnb_train.groupby(['day','year']).agg({'price' : 'mean'}).unstack()
abnbtr_weekly

price            
year             2017        2018
day                              
Friday     226.985632  254.028509
Monday     212.082204  242.697468
Saturday   231.156204  255.199559
Sunday     216.217581  245.339043
Thursday   217.649962  244.357383
Tuesday    212.180119  240.721096
Wednesday  210.919051  237.212299

In [112]:
abnbtr_weekly.reset_index(inplace = True)
abnbtr_weekly

day       price            
year                   2017        2018
0        Friday  226.985632  254.028509
1        Monday  212.082204  242.697468
2      Saturday  231.156204  255.199559
3        Sunday  216.217581  245.339043
4      Thursday  217.649962  244.357383
5       Tuesday  212.180119  240.721096
6     Wednesday  210.919051  237.212299

In [113]:
abnbtr_weekly.columns

MultiIndex(levels=[[u'price', u'day'], [u'2017', u'2018', u'']],
           codes=[[1, 0, 0], [2, 0, 1]],
           names=[None, u'year'])

In [114]:
abnbtr_weekly

day       price            
year                   2017        2018
0        Friday  226.985632  254.028509
1        Monday  212.082204  242.697468
2      Saturday  231.156204  255.199559
3        Sunday  216.217581  245.339043
4      Thursday  217.649962  244.357383
5       Tuesday  212.180119  240.721096
6     Wednesday  210.919051  237.212299

In [116]:
abnbtr_weekly.set_index('day', inplace = True)
abnbtr_weekly

price            
year             2017        2018
day                              
Friday     226.985632  254.028509
Monday     212.082204  242.697468
Saturday   231.156204  255.199559
Sunday     216.217581  245.339043
Thursday   217.649962  244.357383
Tuesday    212.180119  240.721096
Wednesday  210.919051  237.212299

In [117]:
# https://python-graph-gallery.com/11-grouped-barplot/

bar_width = 0.25
r1 = np.arange(abnbtr_weekly.shape[0])
r2 = [x + bar_width for x in r1]

plt.bar(r1, abnbtr)

In [118]:
r1

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
ytraits = df.columns[1:8]
print("Columns for which boxplot with jitter overlay created: ", ytraits)

for i, ytrait in enumerate(ytraits):
    plt.figure()
    plt.title("Boxplot of class with %s"%ytrait)
    sns.boxplot(x='class', y = df[ytrait], data = df)
    sns.stripplot(x='class', y = df[ytrait], data=df, color="red", jitter=0.2, size=2.5)


### Explore different regression models

Fit a regression model that uses the date as a predictor and predicts the price of an Airbnb rental on that date. In this section, you should ignore all other predictors besides the date. Fit the following models on the training set, and compare the $R^2$ of the fitted models on the test set. Include plots of the fitted models for each method.

*Hint*: You may want to convert the `date` column into a numerical variable by taking the difference in days between each date and the earliest date in the column.

1. Fit simple polynomial models of degree 2, 3, 5, and 8 to the training data. Provide train and test R^2 scores and provide plot(s) of a) the daily averages and b) each model's predictions.

2. You should see that the degree 8 polynomial's predictions are awful- either nonsensically near zero [or far too large] at many input values. This isn't just simple overfitting. What is going wrong in this model, and how might it be addressed?

3. Fit a Smoothing spline model in `pygam` with the smoothness parameter chosen by cross-validation on the training set. Provide the train and test scores of the best-performing model, and plot its predictions. Also plot the predictions of models with more and less smoothing. When plotting the smooths, show a scatterplot of the daily averages on the same axes, for context.

4. Of the models, which performs the best? What is the effect of the tuning parameters (degree and smoothness penalty)?

In [ ]:
#your code here


### 1. Polynomials

In [ ]:
#your code here


_your answer here_


### 2. Degree 8 oddity
_your answer here_

### 3. Smoothing spline model with the smoothness parameter chosen by cross-validation on the training set.

In [ ]:
#your code here


### 4. Effects of parameters
_your answer here_

### Part 1b: Adapting to weekends

It is possible/likely that prices on weekends (Friday nights and Saturday nights) are systematically higher than prices on weekdays, and that including this aspect of the data-generating process in our model will produce a better fit.

5. Create a binary indicator of weekend versus non-weekend, and include it in your best-performing model from above. How does it impact the model's performance?  
*Hint*: include a categorical variable in a GAM by importing and using `pygam.f` in the same way you've used `pygam.s` for continuous variables.


In [ ]:
#your code here


### Part 1c: Going the Distance

You may have noticed from your scatterplots of average price versus day on the training set that there are a few days with abnormally high average prices.

Sort the training data in decreasing order of average price, extracting the 10 most expensive dates. Why are the prices on these days so high? Is it something about the days themselves, noise, an artifact of data collection, or something else?

In [ ]:
#your code here


_your answer here_

### Problem 2: Predicting Airbnb Rental Price Through Listing Features

In this problem, we'll continue our exploration of Airbnb data by predicting price based on listing features. The data can be found in `listings_train.csv` and `listings_test.csv`.

First, visualize the relationship between each of the predictors and the response variable. Does it appear that some of the predictors have a nonlinear relationship with the response variable?

In [ ]:
#your code here

_your answer here_

### Part 2a: Polynomial Regression

Fit the following models on the training set and compare the $R^2$ score of the fitted models on the test set:
    
* Linear regression
* Regression with polynomial basis functions of degree 3 (i.e., basis functions $x$, $x^2$, $x^3$ for each predictor $x$) for quantitative predictors.

In [ ]:
#your code here

### Part 2b: Generalized Additive Model (GAM)

1. Do you see any advantage in fitting an additive regression model to these data, compared to the above models?

1. Fit a GAM to the training set, and compare the test $R^2$ of the fitted model to the above models. You may use a smoothing spline basis function on each predictor, with the same smoothing parameter for each basis function, tuned using cross-validation on the training set.

2. Plot and examine the smooth of each predictor for the fitted GAM. What are some useful insights conveyed by these plots?


_your answer here_


In [ ]:
from pygam import LinearGAM, s, f
from sklearn.preprocessing import LabelEncoder
#your code here

In [ ]:
#your code here

### Part 2c: Putting it All Together
Using the GAM outputs and insights from part 1), discuss the most important variables in the driving prices, and give advide to a frugal traveller hoping to visit Boston.

_your answer here_